In [31]:
from dotenv import load_dotenv
import os
import json
import requests
import pandas as pd

# Load env vars from .env file
load_dotenv()

# Access env vars
BLS_API_KEY  = os.environ.get('BLS_API_KEY')
BLS_ENDPOINT = os.environ.get('BLS_ENDPOINT')

# fetch oews_data from BLS API
def fetch_oews_data(series, **kwargs):
    """
    Pass in a list of BLS timeseries to fetch data and return the series
    in JSON format. Arguments can also be provided as kwargs:
        - startyear (4 digit year)
        - endyear (4 digit year)
        - catalog (True or False)
        - calculations (True or False)
        - annualaverage (True or False)
        - registrationKey (api key from BLS website)
    If the registrationKey is not passed in, this function will use the
    BLS_API_KEY fetched from the environment.
    """
    
    # Error handler for API limited to 25 series per req
    if len(series) < 1 or len(series) > 25:
        raise ValueError("Must pass in between 1 and 25 series ids")
    
    # Create headers and payload post data
    headers = {'Content-Type': 'application/json'}
    payload = {
        'seriesid': series,
        'registrationKey': BLS_API_KEY,
    }

    # Update the payload with the keyword arguments and convert to JSON
    payload.update(kwargs)
    payload = json.dumps(payload)

    # Fetch the response from the BLS API
    response = requests.post(BLS_ENDPOINT, data=payload, headers=headers)
    response.raise_for_status()

    # Parse the JSON result
    result = response.json()
    if result['status'] != 'REQUEST_SUCCEEDED':
        raise Exception(result['message'][0])
    return result


oews_series = ['OEUS250000000000013108211', 'OEUS250000000000013116111', 'OEUS250000000000013201111', 
          'OEUS250000000000013205111', 'OEUS250000000000015122111', 'OEUS250000000000015205111', 
          'OEUS250000000000017207111', 'OEUS250000000000019102911', 'OEUS250000000000019201211',
          'OEUS250000000000019301111', 'OEUS250000000000019303911', 'OEUS250000000000019309411',
          'OEUS250000000000025112311', 'OEUS250000000000025112611', 'OEUS250000000000025112511']

series_occupation = ['Project Management Specialist', 'Market Research Analysts and Marketing Specialists', 'Accountants and Auditors',
                     'Financial and Investment Analysts', 'Computer and Information Research Scientists', 'Data Scientist',
                     'Electrical Engineers', 'Biological Scientists', 'Physicists', 'Economists', 'Psychologists', 'Political Scientists', 
                     'English Language and Literature Teachers, Postsecondary', 'Philosophy and Religion Teachers, Postsecondary', 'History Teachers, Postsecondary']

# create dict associating seriesID to respective occupation
series_to_occupation = dict(zip(oews_series, series_occupation))

# fetch respective
oews_data = fetch_oews_data(oews_series, startyear = 2023, endyear = 2023)

if oews_data:
    # pull series data from inside the Results tree
    oews_series_pull = oews_data.get("Results", {}).get("series", [])

    # create empty results list
    results = []

    # comb through each element in series and pull out specific details
    for item in oews_series_pull:
        series_id = item.get('seriesID')
        occupation = series_to_occupation.get(series_id)

        # pull info one level down in data
        for entry in item.get("data", []):
            results.append({
                "occupation_name": occupation,
                "occupation_seriesid": series_id,
                "salary": entry.get("value"),
                "year": entry.get("year")
            })
        # for name in series_occupation:
        #     results.append({
        #         "occupation": name
        #     })
    
    # Convert to DataFrame for easier handling
    oews_df = pd.DataFrame(results)

# # Run the function
# if __name__ == "__main__":
#     df = get_mass_occupation_data()
#     if df is not None:
#         # print(df) #df.to_csv("massachusetts_salaries.csv", index=False)
#         # print("Data saved to massachusetts_salaries.csv")

In [32]:
print(oews_df)

                                      occupation_name  \
0                       Project Management Specialist   
1   Market Research Analysts and Marketing Special...   
2                            Accountants and Auditors   
3                   Financial and Investment Analysts   
4        Computer and Information Research Scientists   
5                                      Data Scientist   
6                                Electrical Engineers   
7                               Biological Scientists   
8                                          Physicists   
9                                          Economists   
10                                      Psychologists   
11                               Political Scientists   
12  English Language and Literature Teachers, Post...   
13    Philosophy and Religion Teachers, Postsecondary   
14                    History Teachers, Postsecondary   

          occupation_seriesid  salary  year  
0   OEUS250000000000013108211   61320  20

### Connect to DB & Create new table

In [40]:
import config
import psycopg2
from configparser import ConfigParser
from sqlalchemy import create_engine


def config(filename='database.ini', section='postgresql'):
    # create file parser
    parser = ConfigParser()

    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db_params = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db_params[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in {1} file'.format(section, filename))
    
    return db_params


def connect():
    """ Connect to PostgreSQL database server"""
    conn = None
    # read config params
    db_params = config()

    # Create the connection string
    conn_string = f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}"

    try:
        
        engine = create_engine(conn_string)
        
        # establish engine connection
        conn = engine.connect()
        print('Connection successful!')
        
        # create 
        oews_df.to_sql('occupation_salaries', con=conn, if_exists='replace', index=False)
        print("OEWS data inserted into DB")

        conn.autocommit = True
        cursor = conn.cursor()
        
        sql_request = '''select * from occupation_salaries;'''
        cursor.execute(sql_request)
        for i in cursor.fetchall():
            print(i)

        # Close cursor and communication with DB
        cursor.close()
        conn.close()

    # connect to PostgreSQL server
    # print('Connecting to PostgreSQL database...')
    # conn = psycopg2.connect(**db_params)

    # cur = conn.cursor()

    # execute statement to verify connection
    # cur.execute("""CREATE TABLE occupation_salaries(
    #             occupation_id SERIAL PRIMARY KEY,
    #             occupation_name VARCHAR(75) UNIQUE NOT NULL,
    #             occupation_seriesid VARCHAR(30) NOT NULL,
    #             salary SMALLINT NOT NULL);
    #             """)

    # conn = psycopg2.connect(conn_string)

    # # Commit changes to DB
    # # conn.commit()

    # # cursor.execute("SELECT * FROM oews_df")
    
    except (Exception, psycopg2.DatabaseError) as error :
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

if __name__ == '__main__':
    connect()


Connection successful!
OEWS data inserted into DB
'Connection' object has no attribute 'cursor'
Database connection closed.


### Insert OEWS Salary into island table

In [19]:
df.to_sql('oews_df', con=conn, if_exists='replace', index=False)

cursor.execute("SELECT * FROM oews_df")


<bound method NDFrame.to_sql of     year   value                  series_id  \
0   2023   61320  OEUS250000000000013108211   
1   2023   47780  OEUS250000000000013116111   
2   2023   62480  OEUS250000000000013201111   
3   2023   66270  OEUS250000000000013205111   
4   2023  132430  OEUS250000000000015122111   
5   2023   75540  OEUS250000000000015205111   
6   2023   77930  OEUS250000000000017207111   
7   2023   64120  OEUS250000000000019102911   
8   2023   98280  OEUS250000000000019201211   
9   2023   77180  OEUS250000000000019301111   
10  2023   51580  OEUS250000000000019303911   
11  2023   65850  OEUS250000000000019309411   
12  2023   59840  OEUS250000000000025112311   
13  2023   61700  OEUS250000000000025112611   

                                           occupation  
0                       Project Management Specialist  
1   Market Research Analysts and Marketing Special...  
2                            Accountants and Auditors  
3                   Financial and Inve